## Setup Gemma 

In [1]:
# Install Keras 3 last. See https://keras.io/getting_started/ for more details.
!pip install -q -U tensorflow-text
!pip install -q tensorflow-cpu
!pip install -q -U keras-nlp tensorflow-hub
!pip install -q -U keras>=3

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
keras-nlp 0.8.1 requires keras-core, which is not installed.



[notice] A new release of pip is available: 23.0.1 -> 24.2
[notice] To update, run: pip install --upgrade pip



[notice] A new release of pip is available: 23.0.1 -> 24.2
[notice] To update, run: pip install --upgrade pip



[notice] A new release of pip is available: 23.0.1 -> 24.2
[notice] To update, run: pip install --upgrade pip



[notice] A new release of pip is available: 23.0.1 -> 24.2
[notice] To update, run: pip install --upgrade pip


### Choose a Backend:
Keras, a user-friendly high-level deep learning API, is crafted for simplicity and seamless usage across multiple frameworks. With Keras 3, you have the flexibility to execute workflows on one of three backends: TensorFlow, JAX, or PyTorch. In this tutorial, we'll configure the backend to utilize JAX

In [2]:
import jax

jax.devices()

E0000 00:00:1724837162.024958      14 common_lib.cc:778] Could not set metric server port: INVALID_ARGUMENT: Could not find SliceBuilder port 8471 in any of the 0 ports provided in `tpu_process_addresses`="local"
=== Source Location Trace: === 
learning/45eac/tfrc/runtime/common_lib.cc:467
E0828 09:26:02.060062363     121 oauth2_credentials.cc:238]            oauth_fetch: UNKNOWN:C-ares status is not ARES_SUCCESS qtype=A name=metadata.google.internal. is_balancer=0: Domain name not found {created_time:"2024-08-28T09:26:02.060045646+00:00", grpc_status:2}


[TpuDevice(id=0, process_index=0, coords=(0,0,0), core_on_chip=0),
 TpuDevice(id=1, process_index=0, coords=(0,0,0), core_on_chip=1),
 TpuDevice(id=2, process_index=0, coords=(1,0,0), core_on_chip=0),
 TpuDevice(id=3, process_index=0, coords=(1,0,0), core_on_chip=1),
 TpuDevice(id=4, process_index=0, coords=(0,1,0), core_on_chip=0),
 TpuDevice(id=5, process_index=0, coords=(0,1,0), core_on_chip=1),
 TpuDevice(id=6, process_index=0, coords=(1,1,0), core_on_chip=0),
 TpuDevice(id=7, process_index=0, coords=(1,1,0), core_on_chip=1)]

In [3]:
import os
import pandas as pd

# Datasets we use
os.environ["DATASET1"] = "/kaggle/input/gemma-rewrite-nbroad/nbroad-v2.csv"
os.environ["DATASET2"] = "/kaggle/input/llm-prompt-recovery-synthetic-datastore/gemma1000_w7b.csv"
# The Keras 3 distribution API is only implemented for the JAX backend for now
os.environ["KERAS_BACKEND"] = "jax"
# Pre-allocate 90% of TPU memory to minimize memory fragmentation and allocation
# overhead
os.environ["XLA_PYTHON_CLIENT_MEM_FRACTION"] = "0.9"

/tmp/ipykernel_14/3747930928.py:2: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


# Load the model

In [4]:
import keras
import keras_nlp

/usr/local/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [5]:
import jax
import os

# The Keras 3 distribution API is only implemented for the JAX backend for now
os.environ["KERAS_BACKEND"] = "jax"
# Pre-allocate 90% of TPU memory to minimize memory fragmentation and allocation overhead
os.environ["XLA_PYTHON_CLIENT_MEM_FRACTION"] = "0.9"

print(jax.devices())

import keras
keras.distribution.list_devices()

[TpuDevice(id=0, process_index=0, coords=(0,0,0), core_on_chip=0), TpuDevice(id=1, process_index=0, coords=(0,0,0), core_on_chip=1), TpuDevice(id=2, process_index=0, coords=(1,0,0), core_on_chip=0), TpuDevice(id=3, process_index=0, coords=(1,0,0), core_on_chip=1), TpuDevice(id=4, process_index=0, coords=(0,1,0), core_on_chip=0), TpuDevice(id=5, process_index=0, coords=(0,1,0), core_on_chip=1), TpuDevice(id=6, process_index=0, coords=(1,1,0), core_on_chip=0), TpuDevice(id=7, process_index=0, coords=(1,1,0), core_on_chip=1)]


['tpu:0', 'tpu:1', 'tpu:2', 'tpu:3', 'tpu:4', 'tpu:5', 'tpu:6', 'tpu:7']

In [6]:
from keras import layers
from tensorflow import data as tf_data  # For dataset input.

num_gpu=len(keras.distribution.list_devices())
# there are 2 cudas in TPU T4x2 - VMv3-8 has 8
gpu_mesh = keras.distribution.DeviceMesh(
    shape=(1,num_gpu),
    axis_names=["batch", "model"], 
    devices=keras.distribution.list_devices()
)
gpu_mesh

<DeviceMesh shape=(1, 8), axis_names=['batch', 'model']>

In [7]:
# Create a LayoutMap instance
layout = keras.distribution.LayoutMap(device_mesh=gpu_mesh)

# Regex to match against the query, key and value matrices in attention layers
layout["decoder_block.*attention.*(query|key|value)/kernel"] = ("model", None, None)
layout["decoder_block.*attention_output/kernel"] = ("model", None, None)
layout["decoder_block.*ffw_gating.*/kernel"] = (None, "model")
layout["decoder_block.*ffw_linear/kernel"] = ("model", None)

# The rule means that for any weights that match with token_embedding/embeddings'
# will be sharded with model dimensions defined in the mesh (e.g., 2 devices), etc.

# Define the model parallel instance - Distribution that shards model variables.
model_parallel = keras.distribution.ModelParallel(layout_map=layout, batch_dim_name="batch")

# Finally set the distribution globally
keras.distribution.set_distribution(model_parallel)

## Load the data

No training data is provided in this competition, but we know what is the structure of it.
* *original_text*: Text in input to the LLM that needs to be transformed.
* *rewrite_prompt*: Prompt/Instruction that was used in the Gemma to transform *original_text*. This is also the target for this competition.
* rewritten_text: Output text that was generated by the Gemma model.

In [8]:
pd.set_option('display.max_colwidth', 512)
df = pd.read_csv('/kaggle/input/llm-prompt-recovery/train.csv')
df

,id,original_text,rewrite_prompt,rewritten_text
0,-1,"The competition dataset comprises text passages that have been rewritten by the Gemma LLM according to some rewrite_prompt instruction. The goal of the competition is to determine what prompt was used to rewrite each original text. Please note that this is a Code Competition. When your submission is scored, this example test data will be replaced with the full test set. Expect roughly 2,000 original texts in the test set.","Convert this into a sea shanty: """"""The competition dataset comprises text passages that have been rewritten by the Gemma LLM according to some rewrite_prompt instruction. The goal of the competition is to determine what prompt was used to rewrite each original text. Please note that this is a Code Competition. When your submission is scored, this example test data will be replaced with the full test set. Expect roughly 2,000 original texts in the test set.""""""","Here is your shanty: (Verse 1) The text is rewritten, the LLM has spun, With prompts so clever, they've been outrun. The goal is to find, the prompt so bright, To crack the code, and shine the light. (Chorus) Oh, this is a code competition, my dear, With text and prompts, we'll compete. Two thousand texts, a challenge grand, To guess the prompts, hand over hand.(Verse 2) The original text, a treasure lost, The rewrite prompt, a secret to be"


We use external data that uses Gemma for transforming text using a prompt. Let's take a look at this data:

This is the template that our model expects

In [9]:
template = (
    "<start_of_turn>user\nGenerate a rewrite_prompt that effectively transforms the given original_text into the provided rewritten_text."
    "Capture the essence and context of the content while improving the language, coherence, and expressiveness."
    "Pay attention to detail, clarity, and overall quality in your generated rewrite_prompt."
    "Here is an example sample: original text-" + df.loc[0, 'original_text'] +
    "rewritten_text-" + df.loc[0, 'rewritten_text'] +
    "and this is the right rewrite_prompt-" + df.loc[0, 'rewrite_prompt'] +
    "Now, You will output in text the most suitable rewrite_prompt. For the given original_text- {original_text}" +
    "and rewritten_text- {rewritten_text}" +
    "<end_of_turn>\n<start_of_turn>model\nrewrite_prompt- {rewrite_prompt}"
)

## Load the model

Let's load the model by using keras_nlp library

In [10]:
# Import model from Keras
gemma_lm = keras_nlp.models.GemmaCausalLM.from_preset("gemma_instruct_7b_en")
gemma_lm.summary()

Attaching 'model.safetensors' from model 'keras/gemma/keras/gemma_instruct_7b_en/2' to your Kaggle notebook...


Attaching 'model.safetensors.index.json' from model 'keras/gemma/keras/gemma_instruct_7b_en/2' to your Kaggle notebook...


Attaching 'metadata.json' from model 'keras/gemma/keras/gemma_instruct_7b_en/2' to your Kaggle notebook...


Attaching 'metadata.json' from model 'keras/gemma/keras/gemma_instruct_7b_en/2' to your Kaggle notebook...


Attaching 'task.json' from model 'keras/gemma/keras/gemma_instruct_7b_en/2' to your Kaggle notebook...


...

Model: "gemma_causal_lm"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                  ┃ Output Shape              ┃         Param # ┃ Connected to               ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ padding_mask (InputLayer)     │ (None, None)              │               0 │ -                          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ token_ids (InputLayer)        │ (None, None)              │               0 │ -                          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ gemma_backbone                │ (None, None, 3072)        │   8,537,680,896 │ padding_mask[0][0],        │
│ (GemmaBackbone)               │                           │                 │ token_ids[0][0]            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ token_embedding               │ (None, None, 256000)      │     786,432,000 │ gemma_backbone[0][0]       │
│ (ReversibleEmbedding)         │                           │                 │                            │
└───────────────────────────────┴───────────────────────────┴─────────────────┴────────────────────────────┘

 Total params: 8,537,680,896 (31.81 GB)

 Trainable params: 8,537,680,896 (31.81 GB)

 Non-trainable params: 0 (0.00 B)

In [11]:
# Define paths
MODEL_LORA_WT_PATH = "/kaggle/input/gemma-7b-finetuned-model-weights/model.weights.lora.h5"

In [12]:
# Lora rank enablement... same as before
gemma_lm.backbone.enable_lora(rank=8)

In [13]:
# Limit the input sequence length to 512 (to control memory usage)... same as before
gemma_lm.preprocessor.sequence_length = 512

# Load only the lora weights in...
gemma_lm.backbone.load_lora_weights(MODEL_LORA_WT_PATH)

In [14]:
# Use mixed precision to speed up all training
#keras.mixed_precision.set_global_policy("mixed_bfloat16")

## Inference before finetuning

Let's attempt to pose a question to the Gemma model before fine-tuning it with our dataset, to observe how Gemma responds.

In [15]:
# Take one sample
row = df.iloc[0]

# Generate Prompt using template
prompt = template.format(
    original_text=row.original_text,
    rewritten_text=row.rewritten_text,
    rewrite_prompt="",
)

# Infer
output = gemma_lm.generate(prompt, max_length=2000)
output

'<start_of_turn>user\nGenerate a rewrite_prompt that effectively transforms the given original_text into the provided rewritten_text.Capture the essence and context of the content while improving the language, coherence, and expressiveness.Pay attention to detail, clarity, and overall quality in your generated rewrite_prompt.Here is an example sample: original text-The competition dataset comprises text passages that have been rewritten by the Gemma LLM according to some rewrite_prompt instruction. The goal of the competition is to determine what prompt was used to rewrite each original text.  Please note that this is a Code Competition. When your submission is scored, this example test data will be replaced with the full test set. Expect roughly 2,000 original texts in the test set.rewritten_text-Here is your shanty: (Verse 1) The text is rewritten, the LLM has spun, With prompts so clever, they\'ve been outrun. The goal is to find, the prompt so bright, To crack the code, and shine t

In [16]:
# We see that the model hallucinates!
print("Rewrite Prompt:\n", output.split("\n<start_of_turn>model")[1].strip())

Rewrite Prompt:
 rewrite_prompt- Convert this into a sea shanty.


# Scoring

Here we take a new dataset and we try to evaluate our model

In [17]:
!pip install -Uq /kaggle/input/sentence-transformers-2-4-0/sentence_transformers-2.4.0-py3-none-any.whl


[notice] A new release of pip is available: 23.0.1 -> 24.2
[notice] To update, run: pip install --upgrade pip


In [18]:
from sklearn.metrics.pairwise import cosine_similarity
from sentence_transformers import SentenceTransformer
from tqdm import tqdm
import numpy as np
tqdm.pandas()

In [19]:
test_df =  pd.read_csv("/kaggle/input/llm-prompt-recovery-ground-truth-1/test (1).csv")
test_df = test_df.head(195)

In [20]:
preds = []
for i in range(len(test_df)):
    row = test_df.iloc[i]

    # Generate Prompt using template
    prompt = template.format(
        original_text=row.original_text,
        rewritten_text=row.rewritten_text,
        rewrite_prompt=""
    )

    # Infer
    output = gemma_lm.generate(prompt, max_length=2000)
    pred = output.replace(prompt, "") # remove the prompt from output
    
    # Store predictions
    preds.append(pred)

In [21]:
preds[0]

'Transform this into a gourmet food festival brochure.'

In [22]:
for i, val in enumerate(preds):
    # Check if the current element contains "<start_of_turn>"
    if "<start_of_turn>" in val:
        # Replace the element at index i with the sentence "Improve this text."
        preds[i] = "Improve this text."

In [23]:
df = pd.DataFrame({'pred': preds})
test_df = pd.concat([test_df, df], axis=1)
test_df.head(1)

,id,original_text,rewrite_prompt,rewritten_text,pred
0,40837062,"South Wales Police officers had been pursuing a red Ford Focus around the Cardiff area at about 18:00 BST on Saturday. A stinger was also used.\nMinutes later, the car collided with a Land Rover Freelander on the eastbound side between junctions 30 and 32.\nThe Independent Police Complaints Commission is probing the incident.\nPolice said the Land Rover's driver was cut free but did not suffer serious injuries.\nThe Ford's Gloucester-based driver died.\nAn IPCC spokesman said: ""We were notified by South...",Make the text into a gourmet food festival brochure,"## Gourmet Food Festival Brochure\n\n**Indulge Your Senses at the Exquisite Gourmet Food Festival**\n\n**A Culinary Celebration of the finest flavors**\n\nJoin us at the South Wales Food Festival, a vibrant celebration of the very best in gourmet cuisine. Immerse yourself in a symphony of aromas, textures, and tastes that will tantalize your senses and leave you wanting more.\n\n**The festival features:**\n\n* **Delectable food stalls:** Sample an array of mouth-watering dishes from around the world, in...",Transform this into a gourmet food festival brochure.


In [24]:
def CVScore(test):
    
    scs = lambda row: abs((cosine_similarity(row["actual_embeddings"], row["pred_embeddings"])) ** 3)
    
    model = SentenceTransformer('/kaggle/input/sentence-t5-base-hf/sentence-t5-base')

    test["actual_embeddings"] = test["rewrite_prompt"].progress_apply(lambda x: model.encode(x, normalize_embeddings=True, show_progress_bar=False).reshape(1, -1))
    test["pred_embeddings"] = test["pred"].progress_apply(lambda x: model.encode(x, normalize_embeddings=True, show_progress_bar=False).reshape(1, -1))
    
    test["score"] = test.apply(scs, axis=1)
    
    return np.mean(test['score'])[0][0]
    
print(f"CV Score: {CVScore(test_df)}")

/usr/local/lib/python3.10/site-packages/torch/_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()


  0%|          | 0/195 [00:00<?, ?it/s]

  3%|▎         | 5/195 [00:00<00:04, 43.27it/s]

  5%|▌         | 10/195 [00:00<00:04, 37.96it/s]

  8%|▊         | 15/195 [00:00<00:04, 40.45it/s]

...

 93%|█████████▎| 181/195 [00:04<00:00, 39.97it/s]

 95%|█████████▌| 186/195 [00:04<00:00, 40.10it/s]

 98%|█████████▊| 191/195 [00:04<00:00, 41.38it/s]

100%|██████████| 195/195 [00:04<00:00, 42.15it/s]

CV Score: 0.7861539125442505


In [25]:
test_df.to_csv('test_df_gemma_7b.csv')